## Setup

In [ ]:
# Load packages
using DifferentialEquations
using Plots, StatsPlots
using CSV, DataFrames
using Turing
using LaTeXStrings
using XLSX
using Optim
using StatsBase
using Random
using KernelDensity
using ProgressMeter
using Distributions
using Measures
using GLM

# Setup plots standard
Plots.default(fontfamily = ("computer modern"))

# Set seed 
Random.seed!(42)

In [ ]:
# Add rhs file
include("model_rhs.jl")
    
# Add parameter file
include("model_default_param.jl")

# Add treatment rhs
include("model_rhs_treatment.jl")

# Add function for calculating VAF
include("model_calc_VAF.jl")

# Add model for Turing inference - individual
include("model_infer_dynamics.jl")

# Add model for Turing inference - hierarchical
include("model_infer_dynamics_Hierarchical_Gibbs_DALIAH.jl")

# Add model function for Turing optimisation
include("model_infer_dynamics_post_Hierarchical.jl")

In [ ]:
# Setup initial conditions for master curve
x00 = 1.0*10^5
x10 = 2.5*10^6
x20 = 6.4*10^11
y00 = 1
y10 = 0
y20 = 0
a0 = 8.1*10^2
s0 = 1

# Collect in one vector
u0 = [x00,x10,x20,y00,y10,y20,a0,s0]

# Setup and solve ODEproblem
tspan = (0.0,90*365)
prob = ODEProblem(model_rhs, u0, tspan, p)
sol = solve(prob, TRBDF2(), reltol = 1e-10, abstol = 1e-10, saveat=1)

# Save as named tuple
master_curve = (t = sol.t, x0 = sol[1,:], x1 = sol[2,:], x2 = sol[3,:], y0 = sol[4,:], y1 = sol[5,:], y2 = sol[6,:], 
a = sol[7,:], s = sol[8,:], VAF = sol[6,:]./(sol[3,:]+sol[6,:]))

## Data

In [ ]:
# Setup dataframes
df_D = DataFrame

# Load data
path = "M:/data_cancer/DALIAH/Trine_DALIAH_5y/DALIAH_5y - Export for Julia.csv"
df_D = CSV.read(path, df_D)

# Change data types
df_D[!,:days] = convert.(Float64,df_D[:,:days])
df_D[!,:RUX] = convert.(Float64,df_D[:,:RUX])


df_D

In [ ]:
# Convert to starting at time for first data point - no matter if JAK or treatment
for i in unique(df_D.patientID)
    # Set pID
    pID = i
    
    # Extract minimum number of days - where IFN>0 or JAK>=0
    df_p = df_D[df_D.patientID .== pID,:]

    minDays = minimum(df_p.days[df_p.IFN .> 0  .|| df_p.JAK .>= 0, :])
    
    # Subtract minimum number of days
    df_D.days[df_D.patientID .== pID, :] = df_D.days[df_D.patientID .== pID, :] .- minDays
end
df_D

In [ ]:
# Define new patients
# new_patients = [22,112,132,163,179,200]
# new_patients = [11,20,44,84,85,103,108,111,114,137,139,145,147,149]
new_patients = [6,16,25,27,44,45,82,102,108,122,126,160,179,200]

## Calculate for all patients and all data points

In [ ]:
# Unique patients
unique_patients = new_patients

# Number of patients
P = length(unique_patients)

# Plot for all patients in loop
@showprogress dt=1 desc="Calculating results for all patients and all data points..." for i=1:P 
    # Extract relevant data
    pID = unique_patients[i]
    df_p = df_D[df_D.patientID .== pID, :]
    
    # Extract latest data point
    maxDays = maximum(df_p.days)

    # Extract VAF
    pVAF = df_p.JAK
    
    # Use only data at which VAF is not NaN
    pVAF = pVAF[.!isnan.(pVAF)]
    
    # Length of pVAF
    J = length(pVAF)

    # Choose treatment effect
    effect = "py0dy1"

    # Chooase to estimate initJAK as well
    estInitJAK = true
    
    # Sample in loop over number of data points
    for j=1:J
        # Extract VAF
        pVAF = df_p.JAK
        
        # Use only data at which VAF is not NaN
        pVAF = pVAF[.!isnan.(pVAF)]

        # Use choose how many data points to use
        nData = j
        pVAF = pVAF[1:nData]

        # Instantiate model
        model = model_infer_dynamics_post_Hierarchical(pVAF,df_p,effect,p,master_curve,estInitJAK)

        # Set seed 
        Random.seed!(42)

        # Sample using Turing
        MCMC = sample(model, HMC(0.05, 10), 10500)

        # Convert to data frame
        df_MCMC = DataFrame(MCMC)
        
        path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/DALIAH_5y - MCMC files/Individual/Post Hierarchical/MCMC_py0dy1_p$(pID)_$(j).csv"
        CSV.write(path, df_MCMC)
    end
end

## Plot for all patients and all data points

In [ ]:
# Unique patients
unique_patients = new_patients

# Number of patients
P = length(unique_patients)

# Maximum number of observations
maxObs = 9

# Storage
RMSE_mat = NaN*ones(P,maxObs)
meanCIwidth_mat = NaN*ones(P,maxObs)

# Plot for all patients in loop
@showprogress dt=1 desc="Plotting results for all patients and all data points..." for i=1:P 
    # Extract relevant data
    pID = unique_patients[i]
    df_p = df_D[df_D.patientID .== pID, :]
    
    # Extract latest data point
    maxDays = maximum(df_p.days)

    # Extract VAF
    pVAF = df_p.JAK
    
    # Use only data at which VAF is not NaN
    pVAF = pVAF[.!isnan.(pVAF)]
    
    # Length of pVAF
    J = length(pVAF)

    # Choose treatment effect
    effect = "py0dy1"

    # Choose time for calculating VAF
    pDays = 0:1:75*30.4
    pDays2 = df_p.days[.!isnan.(df_p.JAK)]
    pVAF2 = df_p.JAK[.!isnan.(df_p.JAK)]
    pDays_plot = pDays[1:30:end]

    # Chooase to estimate initJAK as well
    estInitJAK = true
    
    # Plot in loop for all data points
    for j=1:J
        # Extract VAF
        pVAF = df_p.JAK
        
        # Use only data at which VAF is not NaN
        pVAF = pVAF[.!isnan.(pVAF)]

        # Use choose how many data points to use
        nData = j
        pVAF = pVAF[1:nData]

        # Load results
        df_MCMC = DataFrame
        path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/DALIAH_5y - MCMC files/Individual/Post Hierarchical/MCMC_py0dy1_p$(pID)_$(j).csv"
        df_MCMC = CSV.read(path, df_MCMC)
        
        # Remove burn-in
        deleteat!(df_MCMC,collect(1:500))

        # Storage
        VAF_mat = Array{Float64, 2}(undef, length(pDays), length(df_MCMC[:,"iteration"]))
        VAF_mat_points = Array{Float64, 2}(undef, length(pDays2), length(df_MCMC[:,"iteration"]))
        p_mat = Array{Float64, 3}(undef, length(pDays), length(df_MCMC[:,"iteration"]), 8)

        # Calculate VAF in loop
        @showprogress dt=1 desc="Calculating VAF for MCMC chain..." for k in eachindex(df_MCMC.rho1)
            # Set rho values
            rho = [df_MCMC.rho1[k], df_MCMC.rho2[k]]

            # Set initial JAK
            initJAK = df_MCMC.initJAK[k]

            # Calculate VAF using function
            VAF_mat[:,k], sol = model_calc_VAF(rho,df_p,effect,pDays,p,master_curve,initJAK)
            VAF_mat_points[:,k], sol_points = model_calc_VAF(rho,df_p,effect,pDays2,p,master_curve,initJAK)
            p_curve = (t = sol.t, x0 = sol[1,:], x1 = sol[2,:], x2 = sol[3,:], y0 = sol[4,:], y1 = sol[5,:], y2 = sol[6,:], 
                       a = sol[7,:], s = sol[8,:], VAF = sol[6,:]./(sol[3,:]+sol[6,:]))
            p_mat[:,k,:] = [p_curve.x0 p_curve.x1 p_curve.x2 p_curve.y0 p_curve.y1 p_curve.y2 p_curve.a p_curve.s]
        end
        VAF_mat_plot = VAF_mat[1:30:end,:]

        # Calculate RMSE based on mean at the points
        RMSE = rmsd(median(VAF_mat_points, dims=2),df_p.JAK[.!isnan.(df_p.JAK)])
        
        # Save result
        RMSE_mat[i,j] = RMSE
        
        # Calculate mean width of 95% CI
        mean_width = mean(abs.(quantile.(eachrow(VAF_mat), 0.975)*100-quantile.(eachrow(VAF_mat), 0.025)*100))
        
        # Save result
        meanCIwidth_mat[i,j] = mean_width

        # Calculate mean parameter values
        median_param_1 = median(df_MCMC[:,"rho1"])
        median_param_2 = median(df_MCMC[:,"rho2"])
        
        # Plot results
        figVAF = plot(df_p.days/30.4, zeros(size(df_p.days)), fillrange = df_p.IFN, line =:steppost, fillalpha = 0.1, linealpha = 0, 
            linewidth = 3, label = "", colour= :blue1)
        plot!([maxDays/30.4,75], zeros(2), fillrange = [df_p.IFN[end],df_p.IFN[end]], line =:steppost, fillalpha = 0.1, linealpha = 0,
          linewidth = 3, label = "", colour= :blue1)
        title!(L"JAK2"*" VAF for Patient $(pID), RMSE = $(round(RMSE; sigdigits=3)),\n"*L"\rho="
               *"$(round(median_param_1; sigdigits=3)), " * L"\delta="*"$(round(median_param_2; sigdigits=3))")
        xlabel!(L"t"*"/months")
        ylabel!("IFN/"*L"\mathrm{\frac{\mu g}{day}}")
        xlims!(0, 75)
        ylims!(0, 20)

        p2 = twinx()
        plot!(p2, pDays_plot/30.4, quantile.(eachrow(VAF_mat_plot), 0.025)*100, fillrange = quantile.(eachrow(VAF_mat_plot), 0.975)*100, fillalpha = 0.35, 
             colour = :grey60, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
#         plot!(p2, pDays[pDays.<=maxDays]/30.4, mean(VAF_mat[pDays.<=maxDays,:],dims=2)*100, linewidth = 3, 
#                  label = L"JAK2"*" VAF - Mean", colour = :darkgoldenrod1)
#         plot!(p2, pDays[pDays.>maxDays]/30.4, mean(VAF_mat[pDays.>maxDays,:],dims=2)*100, linewidth = 3, 
#               label = L"JAK2"*" VAF - Mean prediction", colour= :darkgoldenrod1, linestyle = :dash)
        plot!(p2, pDays_plot[pDays_plot.<=maxDays]/30.4, median(VAF_mat_plot[pDays_plot.<=maxDays,:],dims=2)*100, linewidth = 3, 
              label = L"JAK2"*" VAF - Median", colour = :darkgoldenrod1)
        plot!(p2, pDays_plot[pDays_plot.>maxDays]/30.4, median(VAF_mat_plot[pDays_plot.>maxDays,:],dims=2)*100, linewidth = 3, 
              label = L"JAK2"*" VAF - Median prediction", colour = :darkgoldenrod1, linestyle = :dash)
        scatter!(p2, pDays2[1:nData]./30.4, pVAF2[1:nData]*100, label=L"\textrm{Data}", markercolor=:red, markersize=4, markerstrokewidth=0.25)
        scatter!(p2, pDays2[nData+1:end]./30.4, pVAF2[nData+1:end]*100, label=L"\textrm{Data}", markercolor=:magenta3, markershape=:diamond, markersize=4, markerstrokewidth=0.25)
        ylabel!(p2, L"JAK2"*" VAF/%")
        xlims!(p2, 0, 75)
        ylims!(p2, 0, 100)
        plot!(titlefont=18,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, legend = :none, margin = 5mm, 
              leftmargin = 8mm, rightmargin = 8mm)
        
        # Save figure
        figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Individual/MCMC - Post Hierarchical/"
        savefig(figVAF,figpath*"Individual_MCMC_$(effect)_post_Hierarchical_$(pID)_$(j).png")
        savefig(figVAF,figpath*"Individual_MCMC_$(effect)_post_Hierarchical_$(pID)_$(j).pdf")
        savefig(figVAF,figpath*"Individual_MCMC_$(effect)_post_Hierarchical_$(pID)_$(j).svg")
        
        # Plot density
        l = @layout [a a; a a; a a; a a;]
        plot_rho1_chain = plot(df_MCMC.iteration,df_MCMC.rho1,linewidth = 0.5, colour = 1, label = "")
        title!("\nChain for "*L"\rho_{s_{y_0}}")
        xlabel!("Iteration")
        ylabel!(L"\rho/1")
        plot_rho1_density = plot(density(df_MCMC.rho1,legend=false),linewidth = 0.5, colour = 1, label = "")
        xlims!(0, 1)
        title!("\nDensity for "*L"\rho")
        xlabel!(L"\rho/1")
        ylabel!("pdf/1")
        plot_rho2_chain = plot(df_MCMC.iteration,df_MCMC.rho2,linewidth = 0.5, colour = 1, label = "")
        title!("Chain for "*L"\delta")
        xlabel!("Iteration")
        ylabel!(L"\rho_{p_{y_1}}/\textrm{day}^{-1}")
        plot_rho2_density = plot(density(df_MCMC.rho2,legend=false),linewidth = 0.5, colour = 1, label = "")
        xlims!(0, 1)
        title!("Density for "*L"\delta")
        xlabel!(L"\rho_{p_{y_1}}/\textrm{day}^{-1}")
        ylabel!("pdf/1")
        plot_initJAK_chain = plot(df_MCMC.iteration,df_MCMC.initJAK*100,linewidth = 0.5, colour = 1, label = "")
        title!("Chain for Initial VAF")
        xlabel!("Iteration")
        ylabel!("Initial VAF/%")
        plot_initJAK_density = plot(density(df_MCMC.initJAK*100,legend=false),linewidth = 0.5, colour = 1, label = "")
        xlims!(0, 100)
        title!("Density for "*"Initial VAF")
        xlabel!("Initial VAF/%")
        ylabel!("pdf/1")
        plot_sigma_chain = plot(df_MCMC.iteration,df_MCMC.sigma*100, linewidth = 0.5, colour = 1, label = "")
        title!("Chain for "*L"\sigma")
        xlabel!("Iteration")
        ylabel!(L"\sigma/\%")
        plot_sigma_density = plot(density((df_MCMC.sigma*100),legend=false),linewidth = 0.5, colour = 1, label = "")
        xlims!(0, 60)
        title!("Density for "*L"\sigma")
        xlabel!(L"\sigma/\%")
        ylabel!("pdf/1")
        figMCMC = plot(plot_rho1_chain,plot_rho1_density,plot_rho2_chain,plot_rho2_density,plot_initJAK_chain,plot_initJAK_density,
            plot_sigma_chain,plot_sigma_density,layout = l,size = (700, 1000))
        
        # Save figure
        figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Individual/MCMC - Post Hierarchical/"
        savefig(figMCMC,figpath*"Chain_Individual_MCMC_$(effect)_post_Hierarchical_$(pID)_$(j).png")
        savefig(figMCMC,figpath*"Chain_Individual_MCMC_$(effect)_post_Hierarchical_$(pID)_$(j).pdf")
        savefig(figMCMC,figpath*"Chain_Individual_MCMC_$(effect)_post_Hierarchical_$(pID)_$(j).svg")
    end
end

In [ ]:
# Plot all data points for a patient in same figure

# Create empty plots
plot1 = plot()
plot2 = plot()
plot3 = plot()
plot4 = plot()
plot5 = plot()
plot6 = plot()
plot7 = plot()
plot8 = plot()
plot9 = plot()

# Unique patients
unique_patients = new_patients

# Number of patients
P = length(unique_patients)

# Maximum number of observations
maxObs = 9

# Plot for all patients in loop
@showprogress dt=1 desc="Plotting results for all patients and all data points..." for i=11:11 
    # Extract relevant data
    pID = unique_patients[i]
    df_p = df_D[df_D.patientID .== pID, :]
    
    # Extract latest data point
    maxDays = maximum(df_p.days)

    # Extract VAF
    pVAF = df_p.JAK
    
    # Use only data at which VAF is not NaN
    pVAF = pVAF[.!isnan.(pVAF)]
    
    # Length of pVAF
    J = length(pVAF)

    # Choose treatment effect
    effect = "py0dy1"

    # Choose time for calculating VAF
    pDays = 0:1:75*30.4
    pDays2 = df_p.days[.!isnan.(df_p.JAK)]
    pVAF2 = df_p.JAK[.!isnan.(df_p.JAK)]
    pDays_plot = pDays[1:30:end]

    # Chooase to estimate initJAK as well
    estInitJAK = true
    
    # Plot in loop for all data points
    for j=1:J
        # Extract VAF
        pVAF = df_p.JAK
        
        # Use only data at which VAF is not NaN
        pVAF = pVAF[.!isnan.(pVAF)]

        # Use choose how many data points to use
        nData = j
        pVAF = pVAF[1:nData]

        # Load results
        df_MCMC = DataFrame
        path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/DALIAH_5y - MCMC files/Individual/Post Hierarchical/MCMC_py0dy1_p$(pID)_$(j).csv"
        df_MCMC = CSV.read(path, df_MCMC)
        
        # Remove burn-in
        deleteat!(df_MCMC,collect(1:500))

        # Storage
        VAF_mat = Array{Float64, 2}(undef, length(pDays), length(df_MCMC[:,"iteration"]))
        VAF_mat_points = Array{Float64, 2}(undef, length(pDays2), length(df_MCMC[:,"iteration"]))
        p_mat = Array{Float64, 3}(undef, length(pDays), length(df_MCMC[:,"iteration"]), 8)

        # Calculate VAF in loop
        @showprogress dt=1 desc="Calculating VAF for MCMC chain..." for k in eachindex(df_MCMC.rho1)
            # Set rho values
            rho = [df_MCMC.rho1[k], df_MCMC.rho2[k]]

            # Set initial JAK
            initJAK = df_MCMC.initJAK[k]

            # Calculate VAF using function
            VAF_mat[:,k], sol = model_calc_VAF(rho,df_p,effect,pDays,p,master_curve,initJAK)
            VAF_mat_points[:,k], sol_points = model_calc_VAF(rho,df_p,effect,pDays2,p,master_curve,initJAK)
            p_curve = (t = sol.t, x0 = sol[1,:], x1 = sol[2,:], x2 = sol[3,:], y0 = sol[4,:], y1 = sol[5,:], y2 = sol[6,:], 
                       a = sol[7,:], s = sol[8,:], VAF = sol[6,:]./(sol[3,:]+sol[6,:]))
            p_mat[:,k,:] = [p_curve.x0 p_curve.x1 p_curve.x2 p_curve.y0 p_curve.y1 p_curve.y2 p_curve.a p_curve.s]
        end
        VAF_mat_plot = VAF_mat[1:30:end,:]

        # Calculate RMSE based on mean at the points
        RMSE = rmsd(mean(VAF_mat_points, dims=2),df_p.JAK[.!isnan.(df_p.JAK)])
         
        # Calculate mean width of 95% CI
        mean_width = mean(abs.(quantile.(eachrow(VAF_mat), 0.975)*100-quantile.(eachrow(VAF_mat), 0.025)*100))
        
        # Calculate mean parameter values
        median_param_1 = median(df_MCMC[:,"rho1"])
        median_param_2 = median(df_MCMC[:,"rho2"])
        
        # Load results
        df_MCMC = DataFrame
        path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/DALIAH_5y - MCMC files/Individual/Pre Hierarchical/MCMC_py0dy1_p$(pID)_$(j).csv"
        df_MCMC = CSV.read(path, df_MCMC)
        
        # Remove burn-in
        deleteat!(df_MCMC,collect(1:500))

        # Storage
        VAF_mat_pre = Array{Float64, 2}(undef, length(pDays), length(df_MCMC[:,"iteration"]))
        VAF_mat_points_pre = Array{Float64, 2}(undef, length(pDays2), length(df_MCMC[:,"iteration"]))
        p_mat_pre = Array{Float64, 3}(undef, length(pDays), length(df_MCMC[:,"iteration"]), 8)

        # Calculate VAF in loop - pre hierarchical
        @showprogress dt=1 desc="Calculating VAF for MCMC chain..." for k in eachindex(df_MCMC.rho1)
            # Set rho values
            rho = [df_MCMC.rho1[k], df_MCMC.rho2[k]]

            # Set initial JAK
            initJAK = df_MCMC.initJAK[k]

            # Calculate VAF using function
            VAF_mat_pre[:,k], sol = model_calc_VAF(rho,df_p,effect,pDays,p,master_curve,initJAK)
            VAF_mat_points_pre[:,k], sol_points = model_calc_VAF(rho,df_p,effect,pDays2,p,master_curve,initJAK)
            p_curve = (t = sol.t, x0 = sol[1,:], x1 = sol[2,:], x2 = sol[3,:], y0 = sol[4,:], y1 = sol[5,:], y2 = sol[6,:], 
                       a = sol[7,:], s = sol[8,:], VAF = sol[6,:]./(sol[3,:]+sol[6,:]))
            p_mat_pre[:,k,:] = [p_curve.x0 p_curve.x1 p_curve.x2 p_curve.y0 p_curve.y1 p_curve.y2 p_curve.a p_curve.s]
        end
        VAF_mat_plot_pre = VAF_mat_pre[1:30:end,:]
        
        # Plot results
        figVAF = plot(df_p.days/30.4, zeros(size(df_p.days)), fillrange = df_p.IFN, line =:steppost, fillalpha = 0.1, linealpha = 0, 
            linewidth = 3, label = "", colour= :blue1)
        plot!([maxDays/30.4,75], zeros(2), fillrange = [df_p.IFN[end],df_p.IFN[end]], line =:steppost, fillalpha = 0.1, linealpha = 0,
          linewidth = 3, label = "", colour= :blue1)
        if j==1
            title!(L"JAK2"*" VAF for Patient $(pID) using \n the first data point, RMSE = $(round(RMSE; sigdigits=3)),\n "*L"\rho="
                   *"$(round(median_param_1; sigdigits=3)), " * L"\delta="*"$(round(median_param_2; sigdigits=3))")
        else
            title!(L"JAK2"*" VAF for Patient $(pID) using \n the first $(j) data points, RMSE = $(round(RMSE; sigdigits=3)),\n "*L"\rho="
               *"$(round(median_param_1; sigdigits=3)), " * L"\delta="*"$(round(median_param_2; sigdigits=3))")
        end
        xlabel!(L"t"*"/months")
        ylabel!("IFN/"*L"\mathrm{\frac{\mu g}{day}}")
        xlims!(0, 75)
        ylims!(0, 20)
        p2 = twinx()
        plot!(p2, pDays_plot/30.4, quantile.(eachrow(VAF_mat_plot), 0.025)*100, fillrange = quantile.(eachrow(VAF_mat_plot), 0.975)*100, fillalpha = 0.35, 
             colour = :grey60, label = L"\textrm{95\%\ Credibility\ Interval}", linealpha = 0)
#         plot!(p2, pDays[pDays.<=maxDays]/30.4, mean(VAF_mat[pDays.<=maxDays,:],dims=2)*100, linewidth = 3, 
#                  label = L"JAK2"*" VAF - Mean", colour = :darkgoldenrod1)
#         plot!(p2, pDays[pDays.>maxDays]/30.4, mean(VAF_mat[pDays.>maxDays,:],dims=2)*100, linewidth = 3, 
#               label = L"JAK2"*" VAF - Mean prediction", colour= :darkgoldenrod1, linestyle = :dash)
        plot!(p2, pDays_plot[pDays_plot.<=maxDays]/30.4, median(VAF_mat_plot_pre[pDays_plot.<=maxDays,:],dims=2)*100, linewidth = 6, 
              label = L"JAK2"*" VAF - Median", colour = :blue1)
        plot!(p2, pDays_plot[pDays_plot.>maxDays]/30.4, median(VAF_mat_plot_pre[pDays_plot.>maxDays,:],dims=2)*100, linewidth = 6, 
              label = L"JAK2"*" VAF - Median prediction", colour = :blue1, linestyle = :dash)
        plot!(p2, pDays_plot[pDays_plot.<=maxDays]/30.4, median(VAF_mat_plot[pDays_plot.<=maxDays,:],dims=2)*100, linewidth = 6, 
              label = L"JAK2"*" VAF - Median", colour = :darkgoldenrod1)
        plot!(p2, pDays_plot[pDays_plot.>maxDays]/30.4, median(VAF_mat_plot[pDays_plot.>maxDays,:],dims=2)*100, linewidth = 6, 
              label = L"JAK2"*" VAF - Median prediction", colour = :darkgoldenrod1, linestyle = :dash)
        scatter!(p2, pDays2[1:nData]./30.4, pVAF2[1:nData]*100, label=L"\textrm{Data}", markercolor=:red, markersize=6, markerstrokewidth=0.1)
        scatter!(p2, pDays2[nData+1:end]./30.4, pVAF2[nData+1:end]*100, label=L"\textrm{Data}", markercolor=:magenta3, markershape=:diamond, markersize=6, markerstrokewidth=0.1)
        ylabel!(p2, L"JAK2"*" VAF/%")
        xlims!(p2, 0, 75)
        ylims!(p2, 0, 100)
        if j==1
            plot1 = plot!(titlefont=15,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, legend = :none, margin = 5mm, 
              leftmargin = 8mm, rightmargin = 8mm)
        elseif j==2
            plot2 = plot!(titlefont=15,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, legend = :none, margin = 5mm, 
              leftmargin = 8mm, rightmargin = 8mm)
        elseif j==3
            plot3 = plot!(titlefont=15,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, legend = :none, margin = 5mm, 
              leftmargin = 8mm, rightmargin = 8mm)
        elseif j==4
            plot4 = plot!(titlefont=15,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, legend = :none, margin = 5mm, 
              leftmargin = 8mm, rightmargin = 8mm)
        elseif j==5
            plot5 = plot!(titlefont=15,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, legend = :none, margin = 5mm, 
              leftmargin = 8mm, rightmargin = 8mm)
        elseif j==6
            plot6 = plot!(titlefont=15,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, legend = :none, margin = 5mm, 
              leftmargin = 8mm, rightmargin = 8mm)
        elseif j==7
            plot7 = plot!(titlefont=15,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, legend = :none, margin = 5mm, 
              leftmargin = 8mm, rightmargin = 8mm)
        elseif j==8
            plot8 = plot!(titlefont=15,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, legend = :none, margin = 5mm, 
              leftmargin = 8mm, rightmargin = 8mm)
        elseif j==9
            plot9 = plot!(titlefont=15,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=10, legend = :none, margin = 5mm, 
              leftmargin = 8mm, rightmargin = 8mm)
        end
    end
end

In [ ]:
# Plot combined figure
figcombined = plot(plot1, plot2, plot3, plot4, plot5, plot6, plot7, plot8, plot9,
                layout=grid(3, 3), link =:x, size = (1600, 1600), margin=5mm)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Individual/MCMC - Post Hierarchical/"
savefig(figcombined,figpath*"Individual_MCMC_py0dy1_post_Hierarchical_126_all.png")
savefig(figcombined,figpath*"Individual_MCMC_py0dy1_post_Hierarchical_126_all.pdf")
savefig(figcombined,figpath*"Individual_MCMC_py0dy1_post_Hierarchical_126_all.svg")

## Compare to results usig uniform priors

In [ ]:
# Load results from pre hierarchical
df_RMSE_vec_pre = DataFrame
path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/RMSE_test_train.csv"
df_RMSE_vec_pre = CSV.read(path, df_RMSE_vec_pre)
df_meanCIwidth_vec_pre = DataFrame
path = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Excel/CI_test_train.csv"
df_meanCIwidth_vec_pre = CSV.read(path, df_meanCIwidth_vec_pre)

# Convert to matrices
RMSE_mat_pre = Matrix(df_RMSE_vec_pre)
meanCIwidth_mat_pre = Matrix(df_meanCIwidth_vec_pre)

In [ ]:
## Storage
mean_RMSE_vec = NaN*ones(maxObs)
mean_meanCIwidth_vec = NaN*ones(maxObs)
mean_RMSE_vec_pre = NaN*ones(maxObs)
mean_meanCIwidth_vec_pre = NaN*ones(maxObs)

# Calculate means in loop
for i=1:maxObs
    temp = RMSE_mat[:,i]
    mean_RMSE_vec[i] = mean(temp[.!(isnan.(temp))])
    temp = meanCIwidth_mat[:,i]
    mean_meanCIwidth_vec[i] = mean(temp[.!(isnan.(temp))])
    temp = RMSE_mat_pre[:,i]
    mean_RMSE_vec_pre[i] = mean(temp[.!(isnan.(temp))])
    temp = meanCIwidth_mat_pre[:,i]
    mean_meanCIwidth_vec_pre[i] = mean(temp[.!(isnan.(temp))])
end

In [ ]:
mean_RMSE_vec

In [ ]:
mean_meanCIwidth_vec

In [ ]:
# Setup for all patients
RMSE_plot = Any[]

# Plot RMSEs in loop
for i=1:length(unique_patients)
    fig1 = plot()
    plot!([1:9], RMSE_mat_pre[i,:], linewidth = 3, label = "Uniform prior", colour= :blue1, xticks = (1:maxObs, 1:maxObs), 
           linestyle = :dash)
    plot!([1:9], RMSE_mat[i,:], linewidth = 3, label = "Population prior", colour= :darkgoldenrod1, xticks = (1:maxObs, 1:maxObs))
    scatter!([1:9], RMSE_mat_pre[i,:], markercolor=:blue1, markersize=4, markerstrokewidth=0.25, label="")
    scatter!([1:9], RMSE_mat[i,:], markercolor=:darkgoldenrod1, markersize=4, markerstrokewidth=0.25, label="")
    title!("Patient $(unique_patients[i])")
    xlabel!("Data points/1")
    ylabel!("RMSE/1")
    plot!(legend=:none,titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=15,
          size = (700, 500), margin=5mm)
    xlims!(1,9)
    if i==9
        ylims!(0,0.5)
    else
        ylims!(0,0.25)
    end
    push!(RMSE_plot,fig1)
end
# Add plot showing the mean
fig1 = plot()
plot!([1:9,], mean_RMSE_vec_pre, linewidth = 3, label = "Uniform prior", colour= :blue1, xticks = (1:maxObs, 1:maxObs), 
      linestyle = :dash)
plot!([1:9,], mean_RMSE_vec, linewidth = 3, label = "Population", colour= :darkgoldenrod1, xticks = (1:maxObs, 1:maxObs))
scatter!([1:9,], mean_RMSE_vec_pre, markercolor=:blue1, markersize=4, markerstrokewidth=0.25, label="")
scatter!([1:9,], mean_RMSE_vec, markercolor=:darkgoldenrod1, markersize=4, markerstrokewidth=0.25, label="")
title!("Mean")
xlabel!("Data points/1")
ylabel!("RMSE/1")
plot!(legend=:none,titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=15,
      size = (700, 500), margin=5mm)
xlims!(1,9)
ylims!(0,0.25)
push!(RMSE_plot,fig1)

# Plot everything
fig_RMSE_all= plot(RMSE_plot..., layout= (3,5), size = (2000, 1000), margin=8mm, leftmargin = 10mm, rightmargin = 8mm)

# Save figure
figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Hierarchical/Gibbs/"
savefig(fig_RMSE_all,figpath*"RMSE_extra_patients.png")
savefig(fig_RMSE_all,figpath*"RMSE_extra_patients.pdf")
savefig(fig_RMSE_all,figpath*"RMSE_extra_patients.svg")

In [ ]:
# Setup for all patients
meanCIwidth_plot = Any[]

# Plot meanCIwidths in loop
for i=1:length(unique_patients)
    fig1 = plot()
    plot!([1:9], meanCIwidth_mat_pre[i,:], linewidth = 3, label = "Uniform prior", colour= :blue1, xticks = (1:maxObs, 1:maxObs), 
           linestyle = :dash)
    plot!([1:9], meanCIwidth_mat[i,:], linewidth = 3, label = "Population prior", colour= :darkgoldenrod1, xticks = (1:maxObs, 1:maxObs))
    scatter!([1:9], meanCIwidth_mat_pre[i,:], markercolor=:blue1, markersize=4, markerstrokewidth=0.25, label="")
    scatter!([1:9], meanCIwidth_mat[i,:], markercolor=:darkgoldenrod1, markersize=4, markerstrokewidth=0.25, label="")
    title!("Patient $(unique_patients[i])")
    xlabel!("Data points/1")
    ylabel!("Width/%")
    plot!(legend=:none,titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=15,
          size = (700, 500), margin=5mm)
    xlims!(1,9)
    ylims!(0,85)
    push!(meanCIwidth_plot,fig1)
end
# Add plot showing the mean
fig1 = plot()
plot!([1:9,], mean_meanCIwidth_vec_pre, linewidth = 3, label = "Uniform prior", colour= :blue1, xticks = (1:maxObs, 1:maxObs), 
      linestyle = :dash)
plot!([1:9,], mean_meanCIwidth_vec, linewidth = 3, label = "Population", colour= :darkgoldenrod1, xticks = (1:maxObs, 1:maxObs))
scatter!([1:9,], mean_meanCIwidth_vec_pre, markercolor=:blue1, markersize=4, markerstrokewidth=0.25, label="")
scatter!([1:9,], mean_meanCIwidth_vec, markercolor=:darkgoldenrod1, markersize=4, markerstrokewidth=0.25, label="")
title!("Mean")
xlabel!("Data points/1")
ylabel!("Width/%")
plot!(legend=:none,titlefont=20,xtickfontsize=15,ytickfontsize=15,xguidefontsize=15,yguidefontsize=15,legendfontsize=15,
      size = (700, 500), margin=5mm)
xlims!(1,9)
ylims!(0,85)
push!(meanCIwidth_plot,fig1)

# Plot everything
fig_meanCIwidth_all= plot(meanCIwidth_plot..., layout= (3,5), size = (2000, 1000), margin=8mm, leftmargin = 8mm, rightmargin = 8mm)

# Save figure
# figpath = "C:/Users/boklund/Documents/Egne artikler m.m/IFN-artikel 2024/Billeder og video/DALIAH_5y Data - Patient Plots/Hierarchical/Gibbs/"
# savefig(fig_meanCIwidth_all,figpath*"meanCIwidth_extra_patients.png")
# savefig(fig_meanCIwidth_all,figpath*"meanCIwidth_extra_patients.pdf")
# savefig(fig_meanCIwidth_all,figpath*"meanCIwidth_extra_patients.svg")

In [ ]:
# Calculate quantities of interest
mean_RMSE_vec

In [ ]:
mean_RMSE_vec_pre

In [ ]:
meanCIwidth_mat

In [ ]:
mean_meanCIwidth_vec

In [ ]:
1 .- mean_RMSE_vec./mean_RMSE_vec_pre

In [ ]:
mean_RMSE_vec_pre .- mean_RMSE_vec